In [1]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

import buffalo.ingestion as ingestion
import buffalo.predictor as predictor
import buffalo.algorithm as algorithm
import buffalo.predictor.models as modeling
import torch
import pickle
import pandas as pd
from buffalo.utility import expand_grid
from tqdm.auto import tqdm

torch.manual_seed(0)

In [2]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [3]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [4]:
target_symbol = 'GE'

In [5]:
ingestor.data['ADJUSTED_DAILY_STOCK'].symbol.unique()

array(['AAPL', 'MSFT', 'IBM', 'META', 'JNJ', 'PFE', 'UNH', 'MARK', 'XLV',
       'JPM', 'BAC', 'GS', 'MS', 'XLF', 'PG', 'KO', 'PEP', 'NKE', 'XLP',
       'GE', 'HON', 'UTX', 'MMM', 'XLI'], dtype=object)

In [6]:
target_stock = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol == @target_symbol')[['open', 'high', 'low', 'adjusted_close', 'volume']]

In [7]:
other_stocks = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol != @target_symbol')

In [8]:
other_stocks

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,symbol,interval,adjusted
time,,,,,,,,,,,
2023-03-31 00:00:00-05:00,162.440002,165.000000,161.910004,164.899994,164.899994,68749792,0.0,1.0,AAPL,daily,1
2023-03-30 00:00:00-05:00,161.529999,162.470001,161.270996,162.360001,162.360001,49501689,0.0,1.0,AAPL,daily,1
2023-03-29 00:00:00-05:00,159.369995,161.050003,159.350006,160.770004,160.770004,51305691,0.0,1.0,AAPL,daily,1
2023-03-28 00:00:00-05:00,157.970001,158.490005,155.979996,157.649994,157.649994,45992152,0.0,1.0,AAPL,daily,1
2023-03-27 00:00:00-05:00,159.940002,160.770004,157.869995,158.279999,158.279999,52390266,0.0,1.0,AAPL,daily,1
...,...,...,...,...,...,...,...,...,...,...,...
1999-11-05 00:00:00-05:00,27.629999,27.860001,27.559999,27.629999,17.832905,88600,0.0,1.0,XLI,daily,1
1999-11-04 00:00:00-05:00,27.750000,27.910000,27.360001,27.480000,17.736092,17800,0.0,1.0,XLI,daily,1
1999-11-03 00:00:00-05:00,27.750000,27.750000,27.559999,27.559999,17.787725,8400,0.0,1.0,XLI,daily,1


In [9]:
for symbol in ['MSFT', 'IBM', 'JNJ', 'PFE', 'UNH', 'XLV', 'JPM', 'BAC', 'GS', 'XLF', 'AAPL', 'KO', 'PEP', 'NKE', 'XLP', 'PG', 'HON', 'MMM', 'XLI']:
    temp = other_stocks[other_stocks['symbol'] == symbol][['open', 'high', 'low', 'adjusted_close', 'volume']].rename(columns={'adjusted_close': 'close'})
    temp.columns = symbol + '_' + temp.columns
    target_stock = predictor.util.align_dataframe_by_time(target_stock, temp)

In [10]:
target_income_statement = ingestor.data['COMPANY_INCOME_STATEMENT'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_balance_sheet = ingestor.data['COMPANY_BALANCE_SHEET'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_cash_flow = ingestor.data['COMPANY_CASH_FLOW'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function', 'net_income']).dropna(axis=1, how='all')
fed_funds_rate = ingestor.data['FEDERAL_FUNDS_RATE'][['value']].rename(columns={'value': 'effective_federal_funds_rate'}).dropna(axis=1, how='all')
payroll = ingestor.data['NONFARM_PAYROLL'][['value']].rename(columns={'value': 'total_nonfarm_payroll'}).dropna(axis=1, how='all')
cpi = ingestor.data['CPI'][['value']].rename(columns={'value': 'consumer_price_index'}).dropna(axis=1, how='all')
unemployment = ingestor.data['UNEMPLOYMENT'][['value']].rename(columns={'value': 'unemployment_rate'}).dropna(axis=1, how='all')
real_gdp = ingestor.data['REAL_GDP'][['value']].rename(columns={'value': 'real_gross_domestic_product'}).dropna(axis=1, how='all')
real_gdp_per_capita = ingestor.data['REAL_GDP_PER_CAPITA'][['value']].rename(columns={'value': 'real_gross_domestic_product_per_capita'})
treasury_yield = ingestor.data['TREASURY_YIELD'][['value', 'maturity']].pivot(columns=['maturity'], values=['value']).dropna(axis=1, how='all')
treasury_yield.columns = 'treasury_yield_' + treasury_yield.columns.droplevel(level=0)

In [11]:
sma = ingestor.data['SMA'].query('symbol == @target_symbol & interval == "daily"')
roc = ingestor.data['ROC'].query('symbol == @target_symbol & interval == "daily"')
ht_sine = ingestor.data['HT_SINE'].query('symbol == @target_symbol & interval == "daily"')
mom = ingestor.data['MOM'].query('symbol == @target_symbol & interval == "daily"')
sma = sma.pivot(columns=['time_period', 'series_type'], values=['sma']).dropna(axis=1, how='all')
sma.columns = sma.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, sma)
roc = roc.pivot(columns=['time_period', 'series_type'], values=['roc']).dropna(axis=1, how='all')
roc.columns = roc.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, roc)
ht_sine = ht_sine.pivot(columns=['time_period', 'series_type'], values=['lead_sine', 'sine']).dropna(axis=1, how='all')
ht_sine.columns = ht_sine.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, ht_sine)
mom = mom.pivot(columns=['time_period', 'series_type'], values=['mom']).dropna(axis=1, how='all')
mom.columns = mom.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, mom)

In [12]:
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, fed_funds_rate)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, payroll)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, cpi)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, unemployment)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp_per_capita)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, treasury_yield)
print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_income_statement)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_balance_sheet)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_cash_flow)
#print(target_stock.shape)

(5692, 160)
(5691, 161)
(5671, 162)
(5671, 163)
(5671, 164)
(5286, 165)


c:\Users\carlo\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 83, using nperseg = 83
  warnings.warn('nperseg = {0:d} is greater than input length '


(5286, 166)
(5246, 171)


In [ ]:
pickle.dump(target_stock, open(r'cached_data/target_stock.pickle', 'wb'))

In [ ]:
target_stock = pickle.load(open(r'cached_data/target_stock.pickle', 'rb'))

In [13]:
target_stock.shape

(5246, 171)

In [14]:
n_head = 1

In [15]:
time_series_data = predictor.util.TimeSeriesData(endog=target_stock[['adjusted_close']], exog=target_stock.drop(columns=['adjusted_close']), seq_len=180, label_len=n_head, name=f'DAILY_ADJUSTED_CLOSE_{target_symbol}')

#### Offline Learning

#### Online Learning

In [16]:
sweep_params = expand_grid(
    hidden_size=[32, 64, 128],
    num_layers=[1, 2, 3, 4],
    dropout=[0.0, 0.2, 0.4],
    batch_size=[32, 64, 128],
    learning_rate=[0.001, 0.005, 0.0001],
    weight_decay=[0.001, 0.0001, 0.00001],
    epochs=[40],
    epochs_per_update=[1, 5, 10, 15],
    update_freq=[1, 5, 10, 15],
    bidirectional = [True, False]
)

In [20]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.RNN(
        input_size=target_stock.shape[1],
        n_ahead=n_head,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=sweep_params.loc[i,'epochs'], epochs_per_update=sweep_params.loc[i,'epochs_per_update'], update_freq=sweep_params.loc[i,'update_freq'], clip_grad_norm_update=None, clip_grad_norm_train=1)
    training_record = predictor.train_and_evaluate_model_online(
        rnn,
        time_series_data,
        update_rule,
        optimizer,
        loss_func,
        train_ratio=0.3,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/online_record.sqlite', additional_note_dataset='', additonal_note_model='')

  0%|          | 0/31104 [00:00<?, ?it/s]

Online training and testing.:   0%|          | 0/3545 [00:00<?, ?it/s]

TypeError: TimeSeriesData.serialize_to_file() got an unexpected keyword argument 'additional_note_dataset'

In [ ]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.LSTM(
        input_size=target_stock.shape[1],
        n_ahead=n_head,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=sweep_params.loc[i,'epochs'], epochs_per_update=sweep_params.loc[i,'epochs_per_update'], update_freq=sweep_params.loc[i,'update_freq'], clip_grad_norm_update=None, clip_grad_norm_train=1)
    training_record = predictor.train_and_evaluate_model_online(
        rnn,
        time_series_data,
        update_rule,
        optimizer,
        loss_func,
        train_ratio=0.3,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/online_record.sqlite', additional_note_dataset='', additonal_note_model='')

In [ ]:
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 2)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()